# Gender Ambiguity Detector

Codebase: https://github.com/sattree/gap

In [28]:
import pandas as pd
import torch
import os
from pathlib import Path
import re

In [29]:
model_version = 'probert'
annotate_coref_mentions = pretrained_proref = False
coref_models = []
sanitize_labels = True
preprocess_eval = True
exp_dir = Path('results/probert')
coref_models_ = []

n_gpu = torch.cuda.device_count()
n_samples = 0

In [30]:
# Prepare model
from models.gap.probert import ProBERT
from models.gap.features import convert_examples_to_features

from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE
from pytorch_pretrained_bert.modeling import BertConfig, WEIGHTS_NAME, CONFIG_NAME
from transformers import BertTokenizer

device = torch.device("cuda" if torch.cuda.is_available() and not False else "cpu")
n_gpu = torch.cuda.device_count()

print('Preparing Model.')
cache_dir = str(PYTORCH_PRETRAINED_BERT_CACHE)
if model_version == 'probert':
    model = ProBERT.from_pretrained("bert-base-uncased",cache_dir=cache_dir,num_labels=2)

model.to(device)

if n_gpu > 1:
    model = torch.nn.DataParallel(model)
        
# Loading the trained model and config that you have fine-tuned
model_name = os.path.abspath("results/probert/train_evaluate/bert-large-uncased/probert/0/model")
output_model_file = os.path.join(model_name, WEIGHTS_NAME)
output_config_file = os.path.join(model_name, CONFIG_NAME)
config = BertConfig(output_config_file)

tokenizer = BertTokenizer.from_pretrained("bert-large-uncased", do_lower_case=True, 
                                          never_split=["[UNK]", "[SEP]", "[PAD]", "[CLS]", "[MASK]"])

if model_version == 'probert':
    model = ProBERT(config,num_labels=2)

if torch.cuda.is_available():
    map_location=lambda storage, loc: storage.cuda()
else:
    map_location='cpu'

model.load_state_dict(torch.load(output_model_file, map_location=map_location))
model.to(device)


Preparing Model.


ProBERT(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )


In [31]:
from tqdm import tqdm
import numpy as np
import contextlib
from torch.utils.data import DataLoader, SequentialSampler,TensorDataset
from models.utils import init_data

def evaluate(model,eval_features,device,eval_mode=True):

    model.eval()

    all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.uint8)
    all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
    all_gpr_tags_mask = torch.tensor([f.gpr_tags_mask for f in eval_features], dtype=torch.uint8)

    all_mention_p_ids = torch.tensor([f.mention_p_ids for f in eval_features], dtype=torch.long)
    all_mention_a_ids = torch.tensor([f.mention_a_ids for f in eval_features], dtype=torch.long)
    all_mention_p_mask = torch.tensor([f.mention_p_mask for f in eval_features], dtype=torch.uint8)
    all_mention_a_mask = torch.tensor([f.mention_a_mask for f in eval_features], dtype=torch.uint8)

    all_cluster_ids_a = torch.tensor([f.cluster_ids_a for f in eval_features], dtype=torch.long)
    all_cluster_mask_a = torch.tensor([f.cluster_mask_a for f in eval_features], dtype=torch.uint8)
    all_cluster_ids_p = torch.tensor([f.cluster_ids_p for f in eval_features], dtype=torch.long)
    all_cluster_mask_p = torch.tensor([f.cluster_mask_p for f in eval_features], dtype=torch.uint8)

    all_pretrained = torch.tensor([f.pretrained for f in eval_features], dtype=torch.float)
    
    all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)

    eval_data = TensorDataset(all_input_ids, 
                                all_input_mask, 
                                all_segment_ids, 
                                all_gpr_tags_mask,
                                all_mention_p_ids,
                                all_mention_a_ids,
                                all_mention_p_mask,
                                all_mention_a_mask,
                                all_cluster_ids_a,
                                all_cluster_mask_a,
                                all_cluster_ids_p,
                                all_cluster_mask_p,
                                all_pretrained,
                                all_label_ids)

    # Run prediction for full data
    eval_sampler = SequentialSampler(eval_data)
    eval_dataloader = DataLoader(eval_data, 
                                sampler=eval_sampler, 
                                batch_size=1)

    eval_loss = 0
    preds = []
    attn_wts = []
    pbar = tqdm(desc="Evaluating", total=len(eval_dataloader)) if eval_mode else contextlib.suppress()
    with pbar:
        for step, batch in enumerate(eval_dataloader):
            # with torch.cuda.device(0):
            batch = tuple(t.to(device) for t in batch)
            (input_ids, input_mask, segment_ids, 
                gpr_tags_mask,
                mention_p_ids, mention_a_ids,
                mention_p_mask, mention_a_mask,
                cluster_ids_a, cluster_mask_a,
                cluster_ids_p, cluster_mask_p, pretrained, label_ids) = batch

            with torch.no_grad():
                res = model(input_ids,
                            segment_ids, 
                            input_mask, 
                            gpr_tags_mask=gpr_tags_mask,
                            mention_p_ids=mention_p_ids,
                            mention_a_ids=mention_a_ids,
                            mention_p_mask=mention_p_mask,
                            mention_a_mask=mention_a_mask, 
                            cluster_ids_a=cluster_ids_a,
                            cluster_mask_a=cluster_mask_a,
                            cluster_ids_p=cluster_ids_p,
                            cluster_mask_p=cluster_mask_p,
                            pretrained=pretrained,
                            labels=None,
                            training=False,
                            eval_mode=eval_mode
                        )

                if eval_mode:
                    logits, probabilties, attn_wts_m, attn_wts_c, attn_wts_co = res
                else:
                    logits, probabilties = res

            if len(preds) == 0:
                preds.append(probabilties.detach().cpu().numpy())
            else:
                preds[0] = np.append(preds[0], probabilties.detach().cpu().numpy(), axis=0)

            if eval_mode:
                pbar.update()

                if len(attn_wts) == 0:
                    attn_wts = [attn_wts_m, attn_wts_c]
                else:
                    attn_wts[0] = np.append(attn_wts[0], attn_wts_m, axis=0)
                    attn_wts[1] = np.append(attn_wts[1], attn_wts_c, axis=0)

    preds = preds[0]
    return preds, attn_wts

## Testing Automatic Pipeline for Using the Detector

In [32]:
# Raw input text
# original_text = ["For the U.S. Under Secretary of State, see Lucy W. Benson. Lucy Benson is a fictional character from the long-running Channel 4 soap opera Hollyoaks," \
#                  "played by Kerrie Taylor between 1995--2000 and appeared in the shows first ever episode airing on the 23 October 1995. The character left five years later when she went travelling.",
#                  "Kathleen Nott was born in Camberwell, London. Her father, Philip, was a lithographic printer, and her mother, Ellen, ran a boarding house in Brixton; Kathleen was their third daughter. "\
#                     "She was educated at Mary Datchelor Girls' School (now closed), London, before attending King's College, London.",
#                 ]    
# original_text = [
#                     "Kathleen Nott was born in Camberwell, London. Her father, Philip, was a lithographic printer, and her mother, Ellen, ran a boarding house in Brixton; Kathleen was their third daughter. "\
#                     "She was educated at Mary Datchelor Girls' School (now closed), London, before attending King's College, London. This is where she met and married them.",
#                 ] 
# original_text = ["Pat (they/them/their) was not sure how they should bring up inclusivity in the workplace."]

In [33]:
# A list of raw input text
original_text = ["Pat (they/them/their) was not sure how they should bring up inclusivity in the workplace.",
                 "For the U.S. Under Secretary of State, see Lucy W. Benson. Lucy Benson is a fictional character from the long-running Channel 4 soap opera Hollyoaks, \
                    played by Kerrie Taylor between 1995--2000 and appeared in the shows first ever episode airing on the 23 October 1995. The character left five years later when she went travelling.",
                "Kathleen Nott was born in Camberwell, London. Her father, Philip, was a lithographic printer, and her mother, Ellen, ran a boarding house in Brixton; Kathleen was their third daughter. \
                    She was educated at Mary Datchelor Girls' School (now closed), London, before attending King's College, London.",
                "This is most notable in two almost unique settings of the Symbolum Apostolorum, a Credo according to the Apostle's Creed, not according to the ordinary of the mass. \
                    Infantas left no conventional mass setting. Michael Noone suggests that, although it possible that Infantas may have been aware of a setting by the French composer Le Brung printed in 1540, it is equally likely that Infantas believed his settings to be unique.",
                "Tilton first made contact with Dozenberg when he was still in Chicago in 1927,\
                    liquidating the print shop. Tilton wrote Dozenberg a letter in Latvian signed simply ``Alfred,'' asking Dozenberg to meet him in New York as soon as he relocated there. \
                        Tilton met with Dozenberg in a restaurant in New York City about a month later and asked Dozenberg to enter his service, without specifying the exact service in question."
                ]

In [34]:
_corenlp_url = 'http://cccxc406.pok.ibm.com:9000/'
from pycorenlp import StanfordCoreNLP
import json
corenlp = StanfordCoreNLP(_corenlp_url)

In [35]:
from typing import List

def is_pronoun(sentence, entity):
    for token_dict in sentence['tokens']:
        if token_dict['originalText'] == entity:
            return token_dict['pos'] == 'PRP' or token_dict['pos'] == 'PRP$'

# need to get combinations of every pronoun + offset with every entity + offset
def get_combinations(entity_dict, pronoun_dict):
    combinations = []
    # each pronoun
    for pronoun in pronoun_dict:
        # offset for each pronoun
        for pronoun_offset in pronoun_dict[pronoun]:
            finished_entities = []
            # each pair of entities
            for entity1 in entity_dict:
                # no duplicates (only want one of [[entity1 = a], [entity2 = b]])
                if entity1 not in finished_entities:
                    # offset for each entity
                    for entity1_offset in entity_dict[entity1]:
                        combinations.append([pronoun, pronoun_offset, entity1, entity1_offset])
                finished_entities.append(entity1)
            
    return combinations

def get_entities_and_pronouns(original_text: List[str]):
    entity_list, pronoun_list = [], []
    if type(original_text) != list and original_text != []:
        raise Exception("Input must be a list of strings.")
    
    for i in range(len(original_text)):
        entity_dict, pronoun_dict = {}, {}
        root = json.loads(corenlp.annotate(original_text[i], properties={'annotators': 'parse,coref,openie,ner', "timeout": "50000"}))

        for sentence_idx in range(len(root['sentences'])):
            sentence = root['sentences'][sentence_idx]
            for idx in range(len(sentence['entitymentions'])):
                entity = sentence['entitymentions'][idx]['ner']
                text = sentence['entitymentions'][idx]['text']
                if entity == 'PERSON' and not is_pronoun(sentence, text):
                    entity_dict[text] = []
            for token_dict in sentence['tokens']:
                if token_dict['pos'] == 'PRP' or token_dict['pos'] == 'PRP$':
                    pronoun_dict[token_dict['originalText']] = []
                    # if is_pronoun(sentence, text):
                    #     pronoun_dict[text] = []
                    # else:
                    #     entity_dict[text] = []

        # add offset from ORIGINAL text 
        # (can't add directly from above because coref annotation adds spaces / other chars) 
        for name in entity_dict:
            if entity_dict[name] == []:
                entity_dict[name] = [word.start() for word in re.finditer(name, original_text[i])]
        for name in pronoun_dict:
            if pronoun_dict[name] == []:
                pronoun_dict[name] = [word.start() for word in re.finditer(name, original_text[i])]
        
        entity_list.append(entity_dict)
        pronoun_list.append(pronoun_dict)
    return entity_list, pronoun_list

entity_list, pronoun_list = get_entities_and_pronouns(original_text)
print(entity_list, pronoun_list)
# combinations = []
# for i in range(len(entity_list)):
#     # get all possible permutations of (entity, pronoun)
#     combs = get_combinations(entity_list[i], pronoun_list[i])
#     # combinations.extend(combs)
#     print(combs)

[{'Pat': [0]}, {'Lucy W. Benson': [43], 'Lucy Benson': [59], 'Kerrie Taylor': [180]}, {'Kathleen Nott': [0], 'Philip': [58], 'Ellen': [110], 'Kathleen': [0, 150]}, {'Infantas': [185, 284, 401], 'Michael Noone': [229], 'Le Brung': [349]}, {'Tilton': [0, 126, 291], 'Dozenberg': [31, 139, 202, 307, 380]}] [{'they': [5, 39], 'them': [10], 'their': [15]}, {'she': [330]}, {'Her': [46], 'her': [53, 98, 105], 'their': [163], 'She': [205]}, {'it': [267, 375], 'his': [1, 419]}, {'he': [42, 46, 111, 247, 261, 432], 'him': [220], 'his': [399]}]


In [36]:
# combinations

In [37]:
# initialize results data frame
cols = ['id', 'text', 'pronoun', 'pronoun_offset', 'a', 'a_offset', 'url']

output_cols = ['id', 'text', 'pronoun', 'pronoun_offset', 'a', 'a_offset', 'url', 'probabilities', 'output']
df_output = pd.DataFrame([], columns=output_cols)

for i in range(len(original_text)):
    combinations = get_combinations(entity_list[i], pronoun_list[i])
    for combination in combinations:
        # each combination is a list with 4 elements 
            # [pronoun, pronoun_offset, entity1, entity1_offset]
        pronoun, pronoun_offset = combination[0], combination[1]
        entity1, entity1_offset = combination[2], combination[3]
        
        new_df = pd.DataFrame([['na',original_text[i], pronoun,pronoun_offset,entity1,entity1_offset,'na']], columns=cols)
        display(new_df)
        
        tmp_write_path = f'{exp_dir}/new_df.csv'
        new_df.to_csv(tmp_write_path, sep='\t', index=False)
        
        # Initialise the data tranformation 
        # Not sure we really need this as all it does is change some columns names then adds 2 new columns 
        X_inference = init_data(exp_dir=exp_dir,test_path=tmp_write_path, verbose=0, mode='inference')
        
        # Tokenisation of the text happens here
        inference_features = convert_examples_to_features(X_inference,tokenizer,512,n_coref_models=0,verbose=0)
        
        # evaluate sample using model
        labels = [True,False]
        predicted_probs, _ = evaluate(model,inference_features,device,eval_mode=True)
        
        # print(predicted_probs)

        # index of max value from predictions so we get exact entity name it resolves to
        for prob in predicted_probs:
            print(prob)
            max_idx = list(prob).index(max(prob))
            print(f"Known pronoun '{pronoun}' resolves '{labels[max_idx]}' to '{entity1}' with a probability of '{prob[max_idx]}'")
        
        output = ""
        if labels[max_idx]:
            output = f"Known pronoun '{pronoun}' resolves to '{entity1}' with a probability of '{prob[max_idx]}'"
        else:
            output = f"Known pronoun '{pronoun}' does not resolve to '{entity1}' with a probability of '{prob[max_idx]}'"
        new_df_output = pd.DataFrame([['na',original_text[i], pronoun,pronoun_offset,entity1,entity1_offset,'na', predicted_probs, output]], columns=output_cols)
        df_output = pd.concat([df_output, new_df_output])
    
df_output.to_csv(f'{exp_dir}/df_output.csv', index=False)

,id,text,pronoun,pronoun_offset,a,a_offset,url
0,na,Pat (they/them/their) was not sure how they sh...,they,5,Pat,0,na


Initializing Step InputReader
Step InputReader initialized
Initializing Step LabelSanitizer
Step LabelSanitizer initialized
Initializing Step MentionsAnnotator
Step MentionsAnnotator initialized
Initializing Step gather_step
STEPPY WARNING: Step with name "InputReader", already exist. Make sure that all Steps have unique name.
Step gather_step initialized
Step gather_step, working in "inference" mode
Step InputReader, working in "inference" mode
Step InputReader, adapting inputs
Step InputReader, transforming...
Step InputReader, transforming completed
Step InputReader, transform completed
Step LabelSanitizer, working in "inference" mode
Step LabelSanitizer, adapting inputs
Step LabelSanitizer, transforming...
Step LabelSanitizer, transforming completed
Step LabelSanitizer, transform completed
Step MentionsAnnotator, working in "inference" mode
Step InputReader, working in "inference" mode
Step InputReader, adapting inputs
Step InputReader, transforming...
Step InputReader, transformin

Transforming data to features.
------BEFORE DATA PIPELINE------- results/probert
Empty DataFrame
Columns: [id, label]
Index: []


Extracting coref clusters...: 100%|██████████| 1/1 [00:00<00:00, 748.45it/s]
Step MentionsAnnotator, transforming completed
Step MentionsAnnotator, transform completed
Step gather_step, adapting inputs
Step gather_step, transforming...
Step gather_step, transforming completed
Step gather_step, transform completed


Transforming data to features done.
 Log a couple of examples for sanity check.



...

Transforming data to features.
------BEFORE DATA PIPELINE------- results/probert
Empty DataFrame
Columns: [id, label]
Index: []


Extracting coref clusters...: 100%|██████████| 1/1 [00:00<00:00, 744.20it/s]
Step MentionsAnnotator, transforming completed
Step MentionsAnnotator, transform completed
Step gather_step, adapting inputs
Step gather_step, transforming...
Step gather_step, transforming completed
Step gather_step, transform completed


Transforming data to features done.
 Log a couple of examples for sanity check.



Convert Examples to features: 1it [00:00, 258.76it/s]
Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]/dccstor/media-bias-4374/gender-ambiguity-detectors/gap/models/gap/probert.py:35: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  ../aten/src/ATen/native/IndexingUtils.h:28.)
  sequence_output = sequence_output[~gpr_tags_mask].view(batch_size, -1, self.config.hidden_size)
Evaluating: 100%|██████████| 1/1 [00:04<00:00,  4.23s/it]

[0.8111978 0.1888022]
Known pronoun 'his' resolves 'True' to 'Dozenberg' with a probability of '0.811197817325592'


In [38]:
pd.read_csv("results/probert/df_output.csv")

,id,text,pronoun,pronoun_offset,a,a_offset,url,probabilities,output
0,na,Pat (they/them/their) was not sure how they sh...,they,5,Pat,0,na,[[0.8394642 0.16053584]],Known pronoun 'they' resolves to 'Pat' with a ...
1,na,Pat (they/them/their) was not sure how they sh...,they,39,Pat,0,na,[[0.7435659 0.2564341]],Known pronoun 'they' resolves to 'Pat' with a ...
2,na,Pat (they/them/their) was not sure how they sh...,them,10,Pat,0,na,[[0.40619457 0.59380543]],Known pronoun 'them' does not resolve to 'Pat'...
3,na,Pat (they/them/their) was not sure how they sh...,their,15,Pat,0,na,[[0.18119642 0.81880355]],Known pronoun 'their' does not resolve to 'Pat...
4,na,"For the U.S. Under Secretary of State, see Luc...",she,330,Lucy W. Benson,43,na,[[0.10031489 0.8996851 ]],Known pronoun 'she' does not resolve to 'Lucy ...
...,...,...,...,...,...,...,...,...,...
116,na,Tilton first made contact with Dozenberg when ...,his,399,Dozenberg,31,na,[[0.12963185 0.8703681 ]],Known pronoun 'his' does not resolve to 'Dozen...
117,na,Tilton first made contact with Dozenberg when ...,his,399,Dozenberg,139,na,[[0.027281 0.972719]],Known pronoun 'his' does not resolve to 'Dozen...
118,na,Tilton first made contact with Dozenberg when ...,his,399,Dozenberg,202,na,[[0.01542503 0.9845749 ]],Known pronoun 'his' does not resolve to 'Dozen...
119,na,Tilton first made contact with Dozenberg when ...,his,399,Dozenberg,307,na,[[0.0845154 0.91548455]],Known pronoun 'his' does not resolve to 'Dozen...
